In [3]:
import utils
df, _ = utils.load_inventory_responses()
df

/Users/will/Code/rust-experiments/notebooks/utils/utils/../../../server/ownership-inventory.log


,id,commitHash,answers,demo,start,end
0,027dde08-15a6-4423-a1d1-558951ca1fa0,c3911952daa4af472cb45e40d59b9da415899758,"[{'question': 'get_or_default', 'answer': {'er...","{'name': 'f', 'email': 'f', 'yearsRust': '1', ...",1675465301072,1675465340963
1,6c789a19-b2f5-4ea3-b83b-bc2f37599609,c3911952daa4af472cb45e40d59b9da415899758,"[{'question': 'apply_curve', 'answer': {'error...","{'email': 'f', 'name': 'f', 'yearsRust': '1', ...",1675465246807,1675465275080
2,96e0263f-692a-4162-9add-31b2e40f65de,c3911952daa4af472cb45e40d59b9da415899758,"[{'question': 'apply_curve', 'answer': {'error...","{'name': 'f', 'email': 'f', 'yearsRust': '1', ...",1675465355671,1675465394346


In [ ]:
import pandas as pd
import json
import textwrap

In [ ]:
with open("../server/ownership-inventory.log", "r") as f:
    logs = []
    for l in f.readlines():
        data = json.loads(l)
        demo = data.pop('demo', None)
        logs.append({
            **demo,
            **data
        })
    logs = pd.DataFrame(logs)

resp = logs.groupby('id').apply(lambda df: df.sort_values('end').iloc[-1])
resp['duration'] = (resp.end - resp.start) / (1000 * 60)

ans = []
for _, row in resp.iterrows():
    row = row.to_dict()
    for a in row.pop('answers', None):
        tasks = a.pop('answer')
        ans.append({
            **row,
            **a,
            **tasks
        })
ans = pd.DataFrame(ans)
ans = ans.groupby(['id', 'question']).apply(lambda df: df.sort_values('end').iloc[0]).reset_index(drop=True)
# ans

In [ ]:
resp.sort_values('start')[['name', 'email', 'duration', 'yearsCpp']].reset_index()

In [ ]:
ans.groupby('question').size()

In [ ]:
for q in ans.question.unique():
    to_delete = ['id', 'start', 'end', 'commitHash', 'duration', 'question']
    ans[ans.question == q].drop(to_delete, axis=1).to_csv(f'data/{q}.csv')

In [ ]:
# for q in ans.question.unique():
#     df = ans[ans.question == q]
#     print('= QUESTION:', q)
    
#     def print_col(k, k_nice):
#         print('==', k_nice)
#         for a in df[k].tolist():
#             print(textwrap.indent(a, '  '))
#             print('  ----')
#         print()
#     print_col('errorExplanation', 'Error explanation:')
#     print_col('messageInterpretation', 'Message interpretation:')
#     print_col('safetyViolation', 'Safety violation:')
#     print_col('functionFix', 'Function fix:')

    
    
#     print('='*30 + '\n')

In [ ]:
import gspread
import gspread_formatting as fmt

gc = gspread.service_account()

sh = gc.open("Ownership inventory data")
all_ws = sh.worksheets()

def find_ws(title):
    for ws in all_ws:
        if ws.title == title:
            return ws

In [ ]:
def ltr(i):
    return chr(65 + i)
assert(ltr(0) == 'A')

In [ ]:
for q in ans.question.unique():
    to_delete = ['id', 'start', 'end', 'commitHash', 'duration', 'question']
    fields = ans[ans.question == q].drop(to_delete, axis=1)    
    cols = fields.columns
    data_i = cols.tolist().index('errorExplanation')

    ws = find_ws(q)
    if not ws:
        ws = sh.add_worksheet(title=q, rows=len(fields)+1, cols=len(cols)+3)
        
    header_fmt = fmt.cellFormat(textFormat=fmt.textFormat(bold=True))
    text_fmt = fmt.cellFormat(wrapStrategy='WRAP', verticalAlignment="TOP")
    code_fmt = fmt.cellFormat(textFormat=fmt.textFormat(fontFamily="Source Code Pro"), verticalAlignment="TOP")
    
    formats = [text_fmt, text_fmt, code_fmt, code_fmt]    
    fmt.format_cell_ranges(ws, [("A1:I1", header_fmt)] + list(zip(map(lambda i: ltr(data_i + i), range(4)), formats)))
    meta_width = 50
    widths = [200, 200, 400, 400]
    fmt.set_column_widths(ws, [(ltr(i), meta_width) for i in range(data_i)] + list(zip(map(lambda i: ltr(data_i + i), range(4)), widths)))
    
    ws.update([cols.tolist()] + fields.values.tolist())